# Scraping YouTube without Comment
Made by Ali Nurdin

In [1]:
# This section will import Selenium and Time
from selenium import webdriver
import time

In [2]:
# This section will open Web Driver
Driver = webdriver.Chrome()

In [29]:
# This section will get you to channel's video tab
Driver.get("https://www.youtube.com/channel/UC-6OyeP_Tk6G-gt5JSVKzkg/videos") # Go to channel video tab
time.sleep(5)

In [30]:
# Videos primary element
Primary = Driver.find_element_by_tag_name("body").find_element_by_id("content").find_element_by_id("page-manager").find_element_by_tag_name("ytd-browse").find_element_by_tag_name("ytd-two-column-browse-results-renderer").find_element_by_id("primary")

In [31]:
# Continuation element
Continuation = Primary.find_element_by_xpath("ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[2]")

In [32]:
# This section defines two functions for finding continuation and skipping the ads
def Continuation(Continuation):
    try:
        Driver.execute_script("return arguments[0].scrollIntoView();", Driver.find_element_by_xpath("html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer/div[2]/yt-next-continuation/paper-spinner"))
        time.sleep(3)
        return True
    except:
        return False
    
def skipAdFunction():
    try:
        time.sleep(4)
        SkipButton = Driver.find_element_by_xpath('//*[@id="movie_player"]').find_element_by_css_selector("div.video-ads.ytp-ad-module").find_element_by_css_selector("div.ytp-ad-player-overlay").find_element_by_css_selector("div.ytp-ad-player-overlay-skip-or-preview").find_element_by_css_selector("div.ytp-ad-skip-ad-slot").find_element_by_css_selector("div.ytp-ad-skip-button-slot").find_element_by_tag_name("span").find_element_by_tag_name("button")
        SkipButton.click()
    except:
        pass
    

In [33]:
# This section will do continuation on the video tab
while Continuation(Continuation):
    continue
    
time.sleep(5)

In [34]:
# This will initialize arrays for video's duration and also video's URL.
Video_duration = []
Video_url = []

In [35]:
# This will append both array above with videos that are in the video tab.
Items = Driver.find_element_by_xpath("html/body/ytd-app/div/ytd-page-manager/ytd-browse/ytd-two-column-browse-results-renderer/div[1]/ytd-section-list-renderer/div[2]/ytd-item-section-renderer/div[3]/ytd-grid-renderer").find_element_by_id("items")
Videos = Items.find_elements_by_tag_name("ytd-grid-video-renderer")

for Video in Videos:
    Driver.execute_script("return arguments[0].scrollIntoView();", Video)
    time.sleep(1)
    Video_duration.append(Video.find_element_by_xpath("div[1]/ytd-thumbnail/a/div[1]/ytd-thumbnail-overlay-time-status-renderer/span").text)
    Video_url.append(Video.find_element_by_xpath("div[1]/ytd-thumbnail/a").get_attribute("href"))

In [36]:
# This will initialize arrays for videos' data.
Videos_data = []

In [55]:
time.sleep(3)
# len(Video_url)
for i in range(0, 1):
    # Go to video Go to the video corresponding to the existing URL.
    Driver.get(Video_url[i])
    time.sleep(4)
    
    # Extract video id
    Video_id = Video_url[i].split("?v=")[-1]
    
    # Find player element
    Column = Driver.find_element_by_tag_name("body").find_element_by_id("columns")
    Primary_Inner = Column.find_element_by_id("primary").find_element_by_id("primary-inner")
    Player = Primary_Inner.find_element_by_id("player")
    
    # Skip the ads
    skipAdFunction()
    
    # Extract video's quality
    Setting_button = Player.find_element_by_xpath("div/div/div/ytd-player/div/div").find_element_by_class_name("ytp-chrome-bottom").find_element_by_xpath("div[2]/div[2]/button[4]") # The index for setting button is either 3 of 4. It's because YouTube add another button which is autoplay button
    Setting_button.click()
    time.sleep(1)
    Menu = Player.find_element_by_xpath("div/div/div/ytd-player/div/div").find_element_by_css_selector("div.ytp-popup.ytp-settings-menu")
    Menu.find_elements_by_class_name("ytp-menuitem")[-1].click() # Quality
    Quality = []
    time.sleep(1)
    for quality in Menu.find_element_by_xpath("div/div[2]").find_elements_by_class_name("ytp-menuitem"):
        Quality.append(quality.text)
    Setting_button.click()
    
    # Find video info such as Title, number of views, and date
    Container = Primary_Inner.find_element_by_id("info").find_element_by_id("info-contents").find_element_by_id("container")
    Title = Container.find_element_by_tag_name("h1").text
    Info = Container.find_element_by_id("info")
    Info_text = Info.find_element_by_id("info-text")
    Number_of_views = Info_text.find_element_by_id("count").find_element_by_css_selector("span.view-count.style-scope.yt-view-count-renderer").text.split(" views")[0]
    Date = Info_text.find_element_by_id("date").find_element_by_tag_name("yt-formatted-string").text.split("Streamed live on ")[-1].split("Premiered ")[-1]
    
    # Number of like and dislike
    like_dislike_number = Info.find_element_by_id("menu-container").find_element_by_id("sentiment").find_element_by_tag_name("paper-tooltip").find_element_by_id("tooltip")
    Driver.execute_script("arguments[0].setAttribute('class','style-scope paper-tooltip')", like_dislike_number)
    number = Info.find_element_by_id("menu-container").find_element_by_id("sentiment").find_element_by_tag_name("paper-tooltip").find_element_by_id("tooltip").text
    Driver.execute_script("arguments[0].setAttribute('class','style-scope paper-tooltip hidden')", like_dislike_number)
    number = number.split(" / ")
    
    # Video's description
    Meta_container = Primary_Inner.find_element_by_id("meta").find_element_by_id("meta-contents").find_element_by_id("container")
    Description = ""
    try:
        Youtube_expander = Meta_container.find_element_by_tag_name("ytd-expander")
        Youtube_expander.find_element_by_id("more").click()
        Description = Youtube_expander.find_element_by_id("content").find_element_by_id("description").find_element_by_tag_name("yt-formatted-string").text
        Youtube_expander.find_element_by_id("less").click()
    except:
        Description = Youtube_expander.find_element_by_id("content").find_element_by_id("description").find_element_by_tag_name("yt-formatted-string").text
    
    # Channel's name and number of its subscribers
    Channel = Meta_container.find_element_by_id("top-row").find_element_by_tag_name("ytd-video-owner-renderer").find_element_by_id("upload-info")
    Channel_name = Channel.find_element_by_id("channel-name").find_element_by_id("text-container").find_element_by_tag_name("a").text
    Number_of_subscriber = Channel.find_element_by_id("owner-sub-count").text.split(" subscribers")[0]
    
    # Number of comments.
    Comments = Primary_Inner.find_element_by_id("comments")
    Driver.execute_script("return arguments[0].scrollIntoView();", Comments)
    time.sleep(2)
    count = Comments.find_element_by_xpath("ytd-item-section-renderer/div[1]/ytd-comments-header-renderer/div[1]/h2/yt-formatted-string").text.split(" Comments")[0]
    
    
    # The video's length
    duration = Video_duration[i]
    seconds = 0
    duration = duration.split(":")
    if(len(duration) == 3):
        seconds = (int(duration[0])*(60**2)) + (int(duration[1])*60) + int(duration[2])
    elif (len(duration) == 2):
        seconds = (int(duration[0])*60) + int(duration[1])
    else:
        seconds = int(duration[0])
    
    Videos_data.append({
        "title_id" : Video_id,
        "title" : Title,
        "date" : Date,
        "description" : Description,
        "duration" : seconds,
        "channel" : Channel_name,
        "number_of_subscriber" : Number_of_subscriber,
        "quality" : Quality,
        "number_of_viewer" : Number_of_views,
        "number_of_like" : number[0],
        "number_of_dislike" : number[1],
        "number_of_comment" : count
    })

In [ ]:
# Importing json
import json

In [ ]:
# This section will enter the data that has been obtained into the json file.
file = open("JSON FILE NAME.json", 'w')
json.dump(Videos_data, file)
file.close()